# Real-Time Object Detection using YOLOv8 (pretrained COCO)

**Author:** Aishwarya

This Colab-ready notebook demonstrates quick setup and real-time inference using a pretrained YOLOv8 model (COCO). It's beginner-friendly and ideal for live demos.

---

Contents:
1. Setup
2. Load pretrained model
3. Inference on images/videos
4. Real-time webcam usage (local)
5. Tips for filtering classes and saving outputs


## 1) Setup

In [ ]:
!pip install -q ultralytics opencv-python-headless

from ultralytics import YOLO
import cv2, time, os
print('Setup complete')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Setup complete


## 2) Load pretrained YOLOv8 model

We'll use `yolov8n.pt` (nano) for speed. It contains COCO classes by default.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
print('Model loaded. Number of classes:', len(model.names))
print(model.names)

Model loaded. Number of classes: 80
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyb

## 3) Inference on an image

Upload an image to the Colab session (left sidebar) or provide a path.

In [ ]:

sample_img = '/content/Bus.png'  # replace with your uploaded image
# If file not present, this cell will show instructions
import os
if not os.path.exists(sample_img):
    print('Upload an image named bus.jpg via the Colab file browser or change the path.')
else:
    results = model.predict(source=sample_img, show=True, conf=0.45)
    print('Done')

Upload an image named bus.jpg via the Colab file browser or change the path.


## 4) Run on a video file (and save output)

Upload a video file named `test_video.mp4` or change the path below.

In [ ]:
# Inference on a video (will save results to runs/detect/predict)
video_path = '/content/istockphoto-2186968774-640_adpp_is.mp4'
import os
if os.path.exists(video_path):
    results = model.predict(source=video_path, conf=0.45, save=True)
    print('Saved results to runs/detect/predict')
else:
    print('Upload test_video.mp4 or change video_path')

Upload test_video.mp4 or change video_path


::## 5) Real-time webcam (local machine)

Colab cannot display a continuous local webcam feed inside the notebook easily. For live demos on your laptop, run the following script locally (not in Colab):

In [ ]:
realtime_code = r"""from ultralytics import YOLO
import cv2, time

model = YOLO('yolov8n.pt')  # ensure the weight file is available locally or will auto-download

cap = cv2.VideoCapture(0)
prev_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame, conf=0.45)
    annotated = results[0].plot()

    # compute FPS
    curr_time = time.time()
    fps = 1.0 / (curr_time - prev_time + 1e-9)
    prev_time = curr_time
    cv2.putText(annotated, f'FPS: {fps:.2f}', (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow('Realtime YOLOv8', annotated)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()"""

with open('realtime_yolo.py', 'w') as f:
    f.write(realtime_code)

print('realtime_yolo.py saved. Download it and run locally for webcam demo.')

realtime_yolo.py saved. Download it and run locally for webcam demo.


## 6) Filter detections by class and save snapshots

Example code to filter for specific classes (e.g., 'person', 'car') and save frames with detections.

In [ ]:
# Example: filter results and save frames with a 'person' detection
from pathlib import Path
Path('snapshots').mkdir(exist_ok=True)
import cv2, numpy as np, os

# process a single image and save if person detected
img_path = '/content/Screenshot 2025-10-28 112928.png'  # replace with your uploaded image
if os.path.exists(img_path):
    res = model(img_path, conf=0.45)
    boxes = res[0].boxes
    found_person = False
    for box in boxes:
        cls = int(box.cls[0])
        name = model.names[cls]
        if name == 'person':
            found_person = True
    if found_person:
        img = cv2.imread(img_path)
        cv2.imwrite('snapshots/detected_person.jpg', img)
        print('Saved snapshots/detected_person.jpg')
    else:
        print('No person found in image.')
else:
    print('Upload or provide an image at img_path to run this cell.')


image 1/1 /content/Screenshot 2025-10-28 112928.png: 448x640 1 bus, 362.9ms
Speed: 16.8ms preprocess, 362.9ms inference, 33.9ms postprocess per image at shape (1, 3, 448, 640)
No person found in image.
